# Week 5 Data Preprocessing: Building Good Training Datasets   

## Unit Convenor & Lecturer  

Elias Maroun

[elias.maroun@mq.edu.au](mailto:elias.maroun@mq.edu.au)

## References    
1. Python Machine Learning 3rd Edition by Raschka & Mirjalili - Chapter 4
2. Various open-source material

## Learning Objectives   

- Issues with missing data
- Types of missingness and their consequences
- Removing and Imputing Missing Values from the Dataset
- Getting Categorical Data into Shape for Use with Machine Learning Algorithms
- Selecting Relevant Features for Model Construction
- Feature Importance


--- 
---
# Missing Data  


If the data is inaccurate, missing or irrelevant we will not be able to produce good predictions (no matter how good and sophisticated our forecasting algorithm is)

- **Quality** and the **amount of useful information** are key determinants of how well a machine learning algorithm can learn
- We must examine and preprocess data before we use it to train a machine learning model





## Dealing with Missing Data   

- Data is often missing
    - Errors in data collection 
    - Certain data doesn't exist 
    - Surveys are incorrectly answered, e.g. income often understated
    - Etc

### Missingness of Data and Its Consequences

- Understanding the **nature of missing data** is crucial in determining how to handle it effectively
- The pattern and mechanism of missing data can significantly impact the outcome of the analysis 



### Types of Missingness

1. **Missing Completely at Random (MCAR)**
- Data is missing entirely at random
- No relationship between the missing data and any observed or unobserved variables
- The likelihood of a data point being missing is the same across all observations
- Example: A survey respondent accidentally skips a question
- Consequences: If data is MCAR, any analysis performed on the data remains unbiased, even if missing data is simply ignored or removed

2. **Missing at Random (MAR)**
- The missingness is related to some of the observed data, but not the missing data itself
- The probability of data being missing is dependent on other observed variables, but not the missing variable itself
- Example: In a medical study, younger patients may be less likely to report their weight, leading to missing data that depends on age but not on weight directly
- Consequences: Data that is MAR can lead to biased results if not handled properly
    - Imputation methods that take other observed variables into account, like multiple imputation or regression-based methods may be appropriate here

3. **Missing Not at Random (MNAR)**
- The missingness is related to the value of the missing data itself  
- The reason data is missing is due to the actual value of the data that is not observed
- Example: In a study on income, people with very high or very low incomes might be less likely to disclose their income
    - This makes the missingness directly related to income
- Consequences: MNAR is the most challenging type of missing data to handle
    - This type of missingness introduces significant bias that simple imputation methods can't address
    - Advanced techniques or domain-specific models are often required,
    - The missing data mechanism needs to be explicitly modeled

---

## Missing Data in Python 

- Common placeholders for missing observations in databases are `NaN` - 'not a number' or `NULL`
- If we try to train an algorithm using such dataframes we will often get an error
- `scikit-learn` was originally developed to work with `NumPy` arrays
    - Clean data in `pandas` and then export into `numpy` using `df.values`
    - Newer versions of some `scikit-learn` libraries also work with `pandas` dataframes

<hr style="width:25%;margin-left:0;">   

### Identifying Missing Values in Tabular Data 

- When dealing with missing data it is easiest to use `pandas`
    - `isnull` method returns a `DataFrame` with Boolean values to indicate whether a cell contains a numeric value (`False`) or if data is missing (`True`)
    - we can also `.sum()` after `isnull` which treats False = 0, and True = 1 -> get a number of missing values for each column or row
    - `pandas` method `.info()` also provides a count of Non-Null Values for each column
    
- Lets create some missing data in a `pandas` DataFrame

```
import pandas as pd
import numpy as np
from io import StringIO # allows us to read from a string as if we are reading from a file

csv_data = 'A, B, C, D \n 1.0, 2.0, 3.0, 4.0 \n 5.0, 6.0,, 8.0 \n 10.0, 11.0, 12.0,'  # note \n inserts a line break (new line)
# print(csv_data, '\n', type(csv_data))

df = pd.read_csv(StringIO(csv_data))
df

```

Now we can detect the missing data
```
df.info()

df.isnull()

print('Missing Values: \n',df.isnull().sum().sort_values())
```

Columns C and D have one missing value each. No other columns have missing values.


<hr style="width:25%;margin-left:0;"> 

### Eliminating Training Examples with Missing Values 

- The easiest but probably **NOT** the best way to deal with missing data is to remove missing observations (either rows or columns)
    - This would be ok if our dataset is large and the data is missing completely at random (MCAR)
    - If the dataset is of moderate or small size then it would lead to a sinficant loss of information

- In `pandas` we can use `dropna()` method
    - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
    - `dropna(axis=1)` - remove missing features (columns)
    - `dropna(axis=0)` - remove missing training examples (rows)
    - Most `pandas` methods have `inplace` parameter 
        - If True, do operation inplace (on our DataFrame) and return None, i.e. change the DataFrame

```
print(df)
print(df.dropna(axis=1)) # drop columns with missing values
print(2*'\n')
print(df)


print(df)
print(df.dropna(axis=0))  # drop rows with missing values
print(2*'\n')
print(df)


print(df)
print(df.dropna(how='all')) # drops rows where all columns are NaN
print(2*'\n')
print(df)


print(df)
print(df.dropna(axis = 0, inplace=True)) # drop rows with missing values and with saves the changes made to df
print(2*'\n')
print(df)
```

---

## Imputing Missing Values 

- *Deleting missing values is not a good option when we have a small dataset or the data is not missing completely at random (MCAR)*
    - Often we have limited amounts of data and need to preserve as much data as we can
    - There are many imputation methods such as for example:
        -  KNN imputation
        -  Predict missing values based on other features, e.g. regression models or decision trees
        -  Forward/backward fill (for time series data)
        -  Mean/Median/Mode Imputation
        -  Etc.

- In **this unit** we are going to consider Mean/Median/Mode Imputation (Interpolation) to fill in missing values
    - This simple method works well in practice for predictive purposes
    - If we are also concerned with model interpretation then more elaborate imputation methods are required
        - Interpretation vs Prediction: if I change x by a certain amount what happens to y vs. for a given value of x what is the prediction of y


### Definitions 
- **Mean** (Average) = $\frac{1}{N}\sum x_i$
- **Median** is the value that separates a dataset into two equal halves
    - It is the middle value when the data points are arranged in ascending or descending order
    - If there is an odd number of data points, the median is the middle value
    - If there is an even number of data points, the median is the average of the two middle values
    - Example: In the dataset 3, 5, 7, 9, 11 the median is 7 (since it's the middle value)
- **Mode** is the most frequently occurring value(s) in a dataset
    - Uni-modal: A dataset can have a single mode if one value occurs more frequently than others.
    - Bi-modal (Multi-model): A dataset is bimodal if two different values occur with the same highest frequency
    - Example: In the dataset 2, 4, 4, 6, 8, 8, 8, 10 the mode is 8 (because it appears three times, more often than any other value)
    - For certain types of data (e.g. nominal, see below) mode is particarly useful since it is impossible to compute the mean or median

### Types of Data in Predictive Analytics

- Which value (mean/median/mode) we for imputation depends on the type of data that is missing
- Variables can be categorised into different types based on their characteristics and how they are represented
- Here are some main types of data in analytics


1. **Categorical Data**
- Data that represent categories or groups
- The values are discrete and often qualitative
    - Nominal: Categories that have no intrinsic order
        - Examples include gender (male, female), color (red, blue, green)
    - Ordinal: Categories that have a clear, ordered relationship
        - Examples include rankings (first, second, third), education levels (high school, bachelor's, master's)

2. **Numerical Data**
- Data that represent numbers and can be used in mathematical operations
    - Continuous: Data that can take any value within a range
        - Data that is measured
        - Examples include height, weight, temperature
    - Discrete: Data that can take specific, distinct values, often integers
        - Usually data that is counted
        - Examples: number of books in a bookstore, number of students in a class, etc.

3. **Text Data**
    - Data in the form of natural language, such as sentences or words
    - Examples: Tweets, reviews, articles

4. **Image Data**
    - Data represented in the form of images, often as pixel values in matrices.
    - Examples: Photographs, medical scans, satellite images

5. **Audio**, **Video**, etc.

---
## Mean/Median/Mode Interpolation    

In our business applications we typically encounter situtations where *categorical* and *numerical* data is missing   
- Mean/Median/Mode Interpolation: Replace missing values with the mean, median, or mode of the column
- However we have to be careful about the type of data we have
    - **Numeric data** -> we typically use mean or median (if concerned about outliers or when the distribution is not symmetrical)    
    - **Categorical data** -> use median or mode (the only value that can be used with nominal data that can't be ordered)   



There are a number of libraries that can help deal with missing data
1. `SimpleImputer` class from `scikit-learn` is quite useful 
    - [https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)
    - Can do `mean`, `median`, `most frequent` and `constant` imputation methods by setting `strategy` parameter
    - `most_frequent` method is useful for categorical feature values, e.g. colour names: red, green blue 
2. `pandas` has a built in `fillna` method
    - [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)


### Imputing with `scikit-learn`

Lets interpolate missing values in our dataframe using **mean imputations** using scikit-learn

```
from sklearn.impute import SimpleImputer
import numpy as np

df = pd.read_csv(StringIO(csv_data)) # --------- read in data with missing observations again ----------
df

# ---------------- sklearn method -------------------------

imr = SimpleImputer(missing_values = np.nan, strategy='mean') # use mean imputation
imr = imr.fit(df)  # .values is used to export pandas dataframe into numpy array

imputed_data = imr.transform(df)
imputed_data # notice that the data is transferred back into a NumPy array



```

<hr style="width:25%;margin-left:0;"> 

### Scikit-learn in more depth {-}

- `SimpleImputer` belongs to the **transformer** class in scikit-learn, which are used for data transformation
- Transformers have two main methods
    - `fit` - used to learn parameters, e.g. column means, from training data
    - `transform` - used learned parameters to transform data
- Any data that is to be transformed must have the same number of features as the dataset that was used to fit the transformer

<img src="images/image1.jpg" alt="Drawing" style="width: 450px;"/>



<!-- ![](images/image2.jpg) -->

### Imputing with `Pandas`

```

# ---------------- pandas method -------------------------

df = pd.read_csv(StringIO(csv_data))  # reset df with missing values #  --------- read in data with missing observations again ----------
df

print(df.mean(axis=0)) 
print(df.fillna(df.mean(axis = 0), inplace=True))  #fill NaN with column mean values
df
```


---
---

# Getting Categorical Data into Shape for Use with Machine Learning Algorithms {-}

- So far we have mainly used **numerical** data as **features** to build predictive models
- When using **categorical** features
    - Need to make sure that such features are encoded correctly 
    - **Ordinal Features** 
        - In order to use classifiers on ordinal data properly we need to convert such variables into **integers**
        - Example: consider shirt sizes: XL > L > M > S  then we would could encode these values as follows: XL = 4, L = 3, M = 2, S = 1   
    - **Nominal Features**:
        - No ordering possible -> we need to encode values using **one-hot encoding** (**dummy variables**)
        - Example: colour: {green, blue, red} see below for encoding 


Consider the following dataset containing:
- a nominal feature (colour)   
- an ordinal feature (size)   
- a numerical feature (price)  


    
```
import pandas as pd

df = pd.DataFrame([
    ['yellow', 'S', 8.2, 'class2'],
    ['green', 'M', 10.1, 'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['blue', 'XL', 15.3, 'class2']])

df.columns = ['colour', 'size', 'price', 'classlabel']
df
```

### Mapping Ordinal Features   

To map ordinal features which are currently encoded as **string** python types into **integers** we need to take care of the ordering of the labels   

- Often we need to do this manually  
- Create a new column whose values reflect ordinal feature labels mapped into integers  
- The easiest way is to create a python **dictionary** and map it into the original column
    - A Python dictionary is a collection of key-value pairs where each key is unique and is used to store and access data
    - They are defined using curly braces {} with keys and values separated by colons
- Here's a short example of creating and accessing a dictionary:

```
my_dict = {'name': 'John', 'age': 30}

print(my_dict['name'])  # Accesses the value associated with key 'name'

my_dict['age'] = 31  # Updates the value of the 'age' key

my_dict
```




- We will use python dictionaries together with the `map` in `pandas`
    - [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html)   
- Used for substituting each value in a Series with another value, that may be derived from a function, a dict or a Series.
- Lets consider the example of shirt sizes

```
df

size_mapping = {'XL':4, 'L':3, 'M':2, 'S':1}
print(size_mapping, type(size_mapping))

df['size2'] = df['size'].map(size_mapping)

df

```

<hr style="width:35%;margin-left:0;">   

### One-Hot Encoding of Nominal Features {-}  

- We must be careful not to encode **nominal features** using integer values that represent an ordering    
    - E.g. red = 1, blue = 2, green = 3
    - This would confuse *linear* classifiers as it will assume that somehow green > blue > red
    - Consider a linear regression using with a positive slope
        - The prediction for blue would always be between the predictions for green and red    


<img src="images/colours.jpg" alt="Drawing" style="width: 450px;"/>
   

One-Hot Encoding and Dummy Variables are both techniques used to convert categorical variables into a numerical format that can be used in machine learning models 
- **One-Hot Encoding**
    - One-hot encoding transforms each category in a categorical variable into a new binary variable (column) with values of 0 or 1
    - Each original category gets its own binary column
    - If a categorical variable has $n$ unique categories, one-hot encoding will create $n$ new columns
    - For each observation/row, the column corresponding to the category will be set to 1, and all other columns will be set to 0


- There are a number of ways to do this
    - `OneHotEncoder` scikit-learn libarary
    - `get_dummies` method from `pandas`   
        - [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)

    
```
df

one_hot = pd.get_dummies(df[['colour']], dtype = int)
print(one_hot)

df2 = df.join(one_hot)
df2

```

- **Dummy Variables**
    - Dummy variables are similar to one-hot encoding but typically drop one category to avoid multicollinearity
        - Especially in the context of linear models
    - The dropped category can be inferred from the remaining categories
    - If a categorical variable has $n$ unique categories, dummy variable encoding will create $n−1$ new columns
    - The dropped category is usually the one that acts as the baseline against which the other categories are compared
 
```
one_hot = pd.get_dummies(df[['colour']], dtype = int, drop_first = True)
print(one_hot)

df3 = df.join(one_hot)
df3

```

<hr style="width:35%;margin-left:0;">   

### Encoding Class Labels

Although most scikit-learn estimators convert class labels (target variable) to integers internally it is best to provide class labels as **integer arrays** to avoid any technical glitches
- **Class labels are not necessarily ordinal variables**
    - E.g. trying to predict sales channels for marketing purposes: Online, In-Store, Mobile App, Call Center, Direct Mail
    - Doesn't matter which integer we assign to a particular label
    - It is easiest to enumerate class labels starting at 0

We can use `LabelEncoder` class from scikit-learn for this  
- Use `fit_transform` method to encode labels  
- Use `inverse_transform` to transform integer labels back to their original string representations
    
```
from sklearn.preprocessing import LabelEncoder

df3

class_le = LabelEncoder()
df3['y'] = class_le.fit_transform(df3['classlabel'].values)
df3
```

### Creating $X$ and $y$ arrays

```
y = df3['y']
y


X = df3[['price', 'size2', 'colour_green', 'colour_red', 'colour_yellow']]
X
```

---  
# Feature Scaling (Again) {-}

Apart from **decision trees** and **random forests** most machine learning algorithms require feature scaling
- Transforming features to the same scale ensures that weights are better optimised    

Consider a random variable $X\sim(\mu,\sigma)$, i.e. $X$ has the mean value of $\mu$ and standard deviation of $\sigma$

- Typically there are two approaches for feature scaling  
1. **Standardization** which we have seen before: if  $X_{stand.} =\frac{X-\mu}{\sigma}$
    - $X_{stand.}$ will have the mean value of 0 and standard deviation of 1, i.e.  $X_{stand.}\sim(0,1)$
    - use `StandardScaler` class from scikit-learn
3. **Normalization** scales features to a range of $[0,1]$ interval
    - E.g. **min-max scaling** $X_{norm}=\frac{X-X_{min}}{X_{max} - X_{min}}$
    - use `MinMaxScalar` class


Which one is more appropriate?  
- It really depends on data properties and should be evaluated on a case-by-case basis  
- Standardization can be more appropriate for optimization algorithms such as gradient descent
- In some cases normalization is more easily interpreted
- Can always try both and see which method produces better forecasts  
    
Lets create some data and see how standardization and normalization compare

```
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

df = pd.DataFrame(range(0, 6), columns=['X'])
df

standard_scaler = StandardScaler()
df['X_standardized'] = standard_scaler.fit_transform(df['X'].values.reshape(-1,1))
df

norm_scaler = MinMaxScaler()
df['X_normalized'] = norm_scaler.fit_transform(df['X'].values.reshape(-1,1))

df
df.plot()

plt.axhline(y=-1, color='gray', linestyle='--', label='y=-1')
plt.axhline(y=0, color='gray', linestyle='--', label='y=0')
plt.axhline(y=1, color='gray', linestyle='--', label='y=1')
plt.show()
```

---
---

# Selecting Meaningful Features & Overfitting (again) {-}   

- Models typically perform better on training datasets than on test datasets. 
    - Why does this happen?  
- Model is overfitted when
    - The model fits to the data in the training dataset too closely, including random variations in the training dataset
    - The model then tries to predict such random variations in the test dataset but because the variations were random in the first place they don't repeat in the test data  
    - The possibility of overfitting arises when models are too complex  
    - Overfitted models are said to have high variance, because the predictions from overfitted models vary a lot  
    - We also say that the model does not generalize well to new data
    - For a more complete discussion see Week 3 lecture notes
- However, poor performance on test data can also happen when the model has not been optimized fully, or the hyperparameters are not selected appropriately


When is a model too complex?    
- In practice this can mean that the model has too many parameters   
    - We are considering too many features/explanatory variables   
    

Possible solutions for overfitting    
- Collect more data and train the model on larger datasets  
- Choose a simpler model with less parameters  
- Introduce a penalty for complexity via regularization  
- Reduce the dimension of the data  

Next we will discuss 

- Regularization (again)
- Dimensionality reduction via feature selection

Both of these techniques will lead to fewer parameters to be fitted to the data and hence reduce the amount of overfitting.

    
    

<hr style="width:35%;margin-left:0;"> 

### L1 and L2 Regularisation as Penalties Against Model Complexity {-}   

In Week 3 we introduced L2 regularisation as a method of reducing model complexity

- Large parameter values are penalised in the cost function
    - This results in **smaller values** of the optimised weights
        - Example: before regularisation $w_1=1.3$, after regularisation $w_1 = 0.7$ 
    - Smaller weights mean that predictor variables have a smaller impact on the forecasts 
    - Weights which were already small before regularization will become zero, eliminating the influence of their corresponding predictors

Typically, we use L1 and L2 regularisations:

- L2 uses the values of squared parameters (this designates 2 in L2)
    - L2: $||w||_2^2=\sum_{j=1}^mw_j^2=w_1^2+w_2^2+\dots+w_m^2$
- L1 regularisation employes absolute values
    - L1: $||w||_1=\sum_{j=1}^m|w_j|=|w_1|+|w_2|+\dots+|w_m|$

**Comparison of L1 and L2 regularizations**

- *Robustness*: L1 is better than L2
    - Robustness is defined as resistance to outliers in a dataset. 
    - The more able a model is to ignore extreme values in the data, the more robust it is. 
    - L1 norm is **more robust** than the L2 norm
    - L2 norm squares values, so it increases the cost of outliers exponentially and hence tries to make large errors associated with outliers smaller
    - L1 norm only takes the absolute value, so it considers them linearly
- *Number of Solutions*: L2 has one solution which is better than L1 that has many solutions
    - Because L2 is Euclidean distance, there is always one right answer as to how to get between two points fastest.
    - L1 is taxicab distance and there are as many solutions. There could be another (mirror image) path of the same length as the blue path.
- *Computational difficulty*: L2 is easier to optimise than L1
    - L2 has a closed form solution because it's using the square function which can be differentiated 
    - L1 does not have a closed form solution because it is a non-differenciable piecewise function, as it involves an absolute value. 
    - For this reason, L1 is computationally more expensive as it mostly rely on approximations (in the lasso case, coordinate descent).
- *Sparsity*: L1 is better for feature selection than L2
    - Sparse matrices or sparse arrays are matrices in which most of the elements are zero. 
    - By contrast, if most of the elements are nonzero, then the matrix is considered dense. 
    - While both L1 and L2 penalties shrink coefficients, L1 tends to shrink coefficients to zero whereas L2 tends to shrink coefficients evenly. 
    - L1 is therefore useful for feature selection, as we can drop any variables associated with coefficients that are estimated to be zero. 
    - This makes a model robust to potentially irrelevant features in the dataset.
    - L2, on the other hand, is useful when we have multicollinearity.
    

---

### Impact of Regularization on Logistic Regression Weights {-}

We'll investigate the impact of regularisation and $C$ (inverse of regularization strength) on Logistic Regression weights 

- Import the wine dataset from [https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data](https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data)   

    - 178 wine examples
    - 13 features describing their different chemical properties
 

```
# import pandas as pd
# import numpy as np

# df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data')


# df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 
#                    'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
#                    'Proline']

# df_wine.to_excel('data/wine.xlsx', index=False)

df_wine = pd.read_excel('data/wine.xlsx')
print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()


```

<hr style="width:35%;margin-left:0;">   

- Use `train_test_split` libarary to split the data into 70% train and 30% test datasets. 
- Stratify according to Class label
    
   
    
```
from sklearn.model_selection import train_test_split

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
```

<hr style="width:35%;margin-left:0;">   
    
- Standardize training and test datasets


```
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()

X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)
```
    

* Normalize training and test datasets (**Normalization** scales features to a range of [0,1] interval)

```
from sklearn.preprocessing import MinMaxScaler

norsc = MinMaxScaler()

X_train_nor = norsc.fit_transform(X_train)
X_test_nor = norsc.transform(X_test)
```

<hr style="width:35%;margin-left:0;">   

- Fit `LogisticRegression` to the training data using L1 penalty (`penalty='l1'`, `C=1.0`, `solver='liblinear'`, `multi_class='ovr'`). 
- Compute accuracy for both training and test datasets.


```
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l1', C=0.1, solver='liblinear', multi_class='ovr')
# Note that C=1.0 is the default. You can increase
# or decrease it to make the regulariztion effect
# stronger or weaker, respectively.

lr.fit(X_train_std, y_train)

print('Training accuracy:', lr.score(X_train_std, y_train))
print('Test accuracy:', lr.score(X_test_std, y_test)) 


# ------- using normalized dataset ---------------

lr_nor = LogisticRegression(penalty='l1', C=0.1, solver='liblinear', multi_class='ovr')

# Note that C=1.0 is the default. You can increase
# or decrease it to make the regulariztion effect
# stronger or weaker, respectively.

lr_nor.fit(X_train_nor, y_train)

print('Training accuracy (normalized):', lr_nor.score(X_train_nor, y_train))
print('Testing accuracy (normalized):', lr_nor.score(X_test_nor, y_test))
```

<hr style="width:35%;margin-left:0;">   

- Print intercepts and weight coefficients of the fitted model

```
print('itercept:',lr.intercept_,'\n')
print('coef:', lr.coef_,'\n')
print("coef's shape:", lr.coef_.shape)    
print(2*'\n')
print('coef not zero:',lr.coef_[lr.coef_!=0],'\n')
print("coef not zero's shape:", lr.coef_[lr.coef_!=0].shape)
```
    

---

### L1 vs L2 Regularization {-}


```
# L1 Regularization VS L2 Regularization
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(8,12))
gs = gridspec.GridSpec(2,1)

# standardized data
ax = fig.add_subplot(gs[0])

colors = ['blue', 'green', 'red', 'cyan', 
          'magenta', 'yellow', 'black', 
          'pink', 'lightgreen', 'lightblue', 
          'gray', 'indigo', 'orange']

weights, params = [], []
for c in range(-4, 6):
    lr = LogisticRegression(penalty='l1', C=10.**c, solver='liblinear', 
                            multi_class='ovr', random_state=0)
    print(10.**c)
    lr.fit(X_train_std, y_train)
    weights.append(lr.coef_[1])
    params.append(10**c)

weights = np.array(weights)

for column, color in zip(range(weights.shape[1]), colors):
    ax.plot(params, weights[:, column],
             label=df_wine.columns[column + 1],
             color=color)

plt.title("Fitted coefficients for different regularisation strengths (L1)",
         fontdict={'weight':'normal','size': 14})
plt.axhline(0, color='black', linestyle='--', linewidth=3)
plt.axvline(x=0.007, color='gray', linestyle='--', label='C=0.007')
plt.axvline(x=0.1, color='gray', linestyle=':', label='C=0.1')

plt.xlim([10**(-5), 10**5])
plt.ylabel('weight coefficient')
plt.xlabel('C')
plt.xscale('log')
plt.legend(loc='upper left')
ax.legend(loc='upper center', bbox_to_anchor=(1.38, 1.03),ncol=1, fancybox=True)

# normalized data
ax = fig.add_subplot(gs[1])

colors = ['blue', 'green', 'red', 'cyan', 
          'magenta', 'yellow', 'black', 
          'pink', 'lightgreen', 'lightblue', 
          'gray', 'indigo', 'orange']

weights, params = [], []
for c in range(-4, 6):
    lr_L2 = LogisticRegression(penalty='l2', C=10.**c, solver='liblinear', 
                            multi_class='ovr', random_state=0)
    print(10.**c)
    lr_L2.fit(X_train_std, y_train)
    weights.append(lr_L2.coef_[1])
    params.append(10**c)

weights = np.array(weights)

for column, color in zip(range(weights.shape[1]), colors):
    ax.plot(params, weights[:, column],
             label=df_wine.columns[column + 1],
             color=color)

plt.title("Fitted weights for different regularization strengths (L2 Regularization)",
         fontdict={'weight':'normal','size': 14})
plt.axhline(0, color='black', linestyle='--', linewidth=3)
plt.axvline(x=0.007, color='gray', linestyle='--', label='C=0.007')
plt.axvline(x=0.1, color='gray', linestyle=':', label='C=0.1')
plt.xlim([10**(-5), 10**5])
plt.ylabel('weight coefficient')
plt.xlabel('C')
plt.xscale('log')
plt.legend(loc='upper left')
ax.legend(loc='upper center', bbox_to_anchor=(1.38, 1.03),ncol=1, fancybox=True)

plt.show()
```

---

# Sequential Feature Selection {-}

An alternative to L1 regularization and sparsity as a method of feature selection is **dimensionality reduction**. There are two main techniques of dimensionality reduction:

1. **Feature Selection** - select a subset of the original features which are relevant to making forecasts
2. **Feature Extraction** - derive information from existing features to construct a new feature subspace 
    - This is done by compressing features in to a new, smaller set of features (will study this next week)

**Feature selection problem:** Out of an initial set of $d$ features choose $k$ most relevant features where $k<d$.  

- E.g. start with $d=100$ features and reduce them to $k=25$ most relevant features.
- Search is typically done by using **sequential** feature selection algorithms
- Sequential selection algorithms are a type of greedy search algorithm

- **Greedy Search Algorithms** - make **locally optimal** choices at each stage of a combinatorial search problem but generally yield a **globally sub-optimal** solution
    - In optimization, the terms "locally optimal" and "globally optimal" refer to the best solutions **within a specific context** or **the entire set of possibilities**, respectively.
    - **Locally Optimal**: A solution is considered locally optimal if it is the best solution within a small surrounding neighborhood. It cannot be improved by making only small changes. However, it is not necessarily the best possible solution across the entire domain of solutions. In complex problems with many variables or in non-linear optimization, there can be many locally optimal solutions.
    - **Globally Optimal**: A solution is globally optimal if it is the best solution across the entire domain of possible solutions. No other feasible solution provides a better outcome. Achieving a globally optimal solution guarantees that there is no other solution that could improve the result, regardless of how significant the change or where in the solution space it occurs.

<br>

- In the graph below, a greedy algorithm is trying to maximize the sum of values inside the circles
    - To do this, it selects the largest number at each step of the algorithm   
    - With a quick visual inspection of the graph, it is clear that this algorithm will not arrive at the correct solution   
    - What is the correct solution?     
    
<img src="images/image8.gif" alt="Drawing" style="width: 450px;"/>
    
<!-- ![](images/image8.jpg) -->

- In contrast **Exhaustive Search Algorithms** - evaluate all possible combinations and are guaranteed to find the optimal solution. However, an exhaustive search is often not computationally feasible.
    - The correct solution for the largest sum is 7,3,1,99.
    - This is clear to us because we can see that no other combination of nodes will come close to a sum of 110  
    - The greedy algorithm fails to solve this problem because it makes decisions purely based on what the best answer at the time is: at each step it did choose the largest number   


**Sequential Backward Selection (SBS)** is a type of sequential feature selection method
- Start by including all possible $d$ features 
- Sequentially remove features one at a time
- In order to determine which feature to remove at each stage define the criterion function $J$, such as the accuracy of classification
- At each step remove a features which results in the least performance loss after its removal


**SBS algorithm**
1. Initialize the algorithm with $k=d$
2. Determine the feature, $x^-$ that maximizes the criterion $x^-=\underset{x}{\text{argmax}}J(X_k-x)$ where $x\in X_d$
    - Note that $J(X_k-x)<J(X_k)$ meaning that the subset $X_k-x$ has lower accuracy that $X_k$
3. Remove the feature $x^-$ from the feature set $X_k$ resulting in $X_{k-1}=X_k-x$
    - E.g. $J$ = accuracy, $J(x_1, x_2, x_3) = 60\%$, $J(x_1, x_2) = 50\%$, $J(x_1, x_3)=40\%$, $J(x_2, x_3)=30\%$ -> choose $J(x_1, x_2) = 50\%$, meaning we eliminated $x_3$ in this step
4. Terminate if $k$ is equal to the number of desired features, otherwise go to step 2.


<hr style="width:35%;margin-left:0;">   

### SBS Algorithm {-}

- First, we'll try the SBS algorithm that comes with the textbook

```
from sklearn.base import clone
from itertools import combinations
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


class SBS():
    def __init__(self, estimator, k_features, scoring=accuracy_score,
                 test_size=0.25, random_state=1):
        self.scoring = scoring
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y):
        
        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=self.test_size,
                             random_state=self.random_state, stratify=y)

        stdsc = StandardScaler()
        X_train = stdsc.fit_transform(X_train)
        X_test = stdsc.transform(X_test)
        
        dim = X_train.shape[1]
        self.indices_ = tuple(range(dim))
        self.subsets_ = [self.indices_]
        score = self._calc_score(X_train, y_train, 
                                 X_test, y_test, self.indices_)
        self.scores_ = [score]

        while dim > self.k_features:
            scores = []
            subsets = []

            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train, y_train, 
                                         X_test, y_test, p)
                scores.append(score)
                subsets.append(p)

            best = np.argmax(scores)
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1

            self.scores_.append(scores[best])
        self.k_score_ = self.scores_[-1]

        return self

    def transform(self, X):
        return X[:, self.indices_]

    def _calc_score(self, X_train, y_train, X_test, y_test, indices):
        self.estimator.fit(X_train[:, indices], y_train)
        y_pred = self.estimator.predict(X_test[:, indices])
        score = self.scoring(y_test, y_pred)
        # score = self.scoring(y_train, y_pred)
        
        return score

```

<hr style="width:35%;margin-left:0;">   

### Feature Selection with SBS algorithm and KNN {-}

We'll use the above SBS algorithm to select a subset of predictors out of 13 features




- Fit the SBS algorithm with KNN on wine training (standarised) data with `k_features = 1`  
- Print optimal subsets starting from 1 to 13 features


```
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

# selecting features
sbs = SBS(knn, k_features=1)

sbs.fit(X, y)

sbs.subsets_
```

<hr style="width:35%;margin-left:0;">   

- Plot the classification accuracy of the KNN classifier for each subset of features

```
k_feat = [len(k) for k in sbs.subsets_]

print(k_feat)
print(sbs.scores_)

plt.plot(k_feat, sbs.scores_, marker='o')
plt.ylim([0.7, 1.02])
plt.ylabel('Accuracy')
plt.xlabel('Number of features')
plt.grid()
plt.tight_layout()
# plt.savefig('images/sbs_knn', dpi=300)
plt.show()
```

<hr style="width:35%;margin-left:0;">   

- Print the smallest feature subset with the classification accuracy of 100%  

```
df_scores = pd.DataFrame(sbs.scores_, columns = ['Scores'])
df_scores['Feature Subsets'] = sbs.subsets_
print(df_scores)

smallest_100_subset = list(df_scores['Feature Subsets'].loc[7])

print(smallest_100_subset)

print('features', df_wine.columns[1:]) # start from 1 since 0 is class label

print(df_wine.columns[1:][smallest_100_subset])
```

### scikit-learn has a Sequential Feature Selection libarary

- Implemented somewhat differently (uses cross-validation which we will cover later in the course)
- Does both `forward` and `backward` feature selection
- Doesn't provide as much information as we have above
- [https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html) 


```
from sklearn.feature_selection import SequentialFeatureSelector

knn2 = KNeighborsClassifier(n_neighbors=5)
sfs = SequentialFeatureSelector(knn2, n_features_to_select=1, direction='backward')
sfs.fit(X, y)

sfs.get_support()
np.arange(X.shape[1])[sfs.get_support()]

```

---
---

# Assessing Feature Importance with Random Forests {-}    

Using a random forest we can measure **feature importance** as the **averaged impurity decrease** computed from all decision trees in the forest
- In scikit-learn access feature importance via 
    - `feature_importances` after fitting `RandomForestClassifier`
    - `SelectFromModel` selects features based on a user-specified importance threshold

A potential problem with feature importance:
- If there is multicollinearity, i.e. features are highly correlated, it is difficult to disentangle importance
    - One feature may rank very high while the other correlated features low (incorrectly)
- Can check correlations between the features to see if multicollinearity is a problem in our application
- If only interested in predictive ability then this is not a problem as we are not trying to interpret our results


<hr style="width:35%;margin-left:0;">   

<span style='background:orange'>  **Exercise 2: Understand the code below** 
    
a) Fit a random forest classifier to the wine data
    
- Note: Random Forest doesn't need data to be standardized   
    
b) Print feature importances  
c) Plot feature importances  
d) Select features with importance greater than 10%  


  

- Fit a random forest classifier to the wine data (note: Random Forest doesn't need data to be standardized)


```
from sklearn.ensemble import RandomForestClassifier

feat_labels = df_wine.columns[1:]

forest = RandomForestClassifier(n_estimators=500, random_state=1)

forest.fit(X_train, y_train)
```


<hr style="width:35%;margin-left:0;">   

- Print feature importances 

```
importances = forest.feature_importances_

# print(importances)  # prints importances of each feature
# print(np.argsort(importances)) # sort the index of importances from smallest to largest
# print(np.argsort(importances)[::-1]) # reverses the index to get largest to smallest


indices = np.argsort(importances)[::-1]

# print(X_train.shape)

for f in range(X_train.shape[1]):
#     print(f, indices[f])   # pick up positions from indices 
    print(f'{f+1:2})  {feat_labels[indices[f]]:40} {importances[indices[f]]:10}')

```

<hr style="width:35%;margin-left:0;">   
 
- Plot feature importances  

```
plt.title('Feature Importance')
plt.bar(range(X_train.shape[1]), 
        importances[indices],
        align='center')

plt.xticks(range(X_train.shape[1]), 
           feat_labels[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
#plt.savefig('images/04_09.png', dpi=300)
plt.show()    
```

<hr style="width:35%;margin-left:0;">   

- Select features with importance greater than 10% 


```
from sklearn.feature_selection import SelectFromModel

sfm = SelectFromModel(forest, threshold=0.1, prefit=True)
X_selected = sfm.transform(X_train)
print('Number of features that meet this threshold criterion:', X_selected.shape[1])

for f in range(X_selected.shape[1]):
    print(f'{f + 1:2}) {feat_labels[indices[f]]:40} {importances[indices[f]]:10}')
    
```